In [1]:
# Import package to be used
import pandas as pd

In [2]:
# Read in data
un_df = pd.read_excel('WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_REV1.xlsx')
un_es = pd.read_excel('WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_REV1_estimate.xlsx')

In [3]:
# Make copy of datasets
df = un_df.copy()
es = un_es.copy()

In [4]:
# Reindex datasets
df.columns = df.iloc[0]
es.columns = df.iloc[0]

In [5]:
# Drop first index and selected columns for historical data
df = df.drop(index=0)
df = df.drop(['Index', 'Variant', 'Notes', 'Location code', 'ISO2 Alpha-code', 'SDMX code**', 'Parent code',
              'Total Population, as of 1 January (thousands)', 'Population Annual Doubling Time (years)',
              'Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)'], axis=1)

In [6]:
# Drop first index and selected columns for estimated data
es = es.drop(index=0)
es = es.drop(['Index', 'Variant', 'Notes', 'Location code', 'ISO2 Alpha-code', 'SDMX code**', 'Parent code',
              'Total Population, as of 1 January (thousands)', 'Population Annual Doubling Time (years)',
              'Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)'], axis=1)

In [7]:
# Rename columns containing highlighted string for historical data
replacers = {', as of 1 July':'', ' (thousands)':'', ' (years)':'', ', both sexes':''}
for i, v in replacers.items():
    df.columns = df.columns.str.replace(i, v, regex=False)

In [8]:
# Rename columns containing highlighted string for estimated population
for i, v in replacers.items():
    es.columns = es.columns.str.replace(i, v, regex=False)

In [9]:
# Preview dataframe
df.head(1)

,"Region, subregion, country or area *",ISO3 Alpha-code,Type,Year,Total Population,Male Population,Female Population,Population Density (persons per square km),Population Sex Ratio (males per 100 females),Median Age,...,"Mortality before Age 60 (deaths under age 60 per 1,000 live births)","Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)","Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)","Mortality between Age 15 and 50 (deaths under age 50 per 1,000 alive at age 15)","Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 50 (deaths under age 50 per 1,000 females alive at age 15)","Male Mortality between Age 15 and 60 (deaths under age 60 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 60 (deaths under age 60 per 1,000 females alive at age 15)",Net Number of Migrants,"Net Migration Rate (per 1,000 population)"
1,WORLD,NaN,World,1950,2499322.157,1245362.965,1253959.192,19.166,99.314,22.224,...,540.385,580.75,498.04,240.316,271.625,208.192,430.259,324.931,0,0


In [10]:
# Rename columns using regular expression for both datasets
df.columns = df.columns.str.replace(r" \(.*\)","", regex=True)
es.columns = es.columns.str.replace(r" \(.*\)","", regex=True)

In [11]:
# Preview dataframe
df.head(1)

,"Region, subregion, country or area *",ISO3 Alpha-code,Type,Year,Total Population,Male Population,Female Population,Population Density,Population Sex Ratio,Median Age,...,Mortality before Age 60,Male Mortality before Age 60,Female Mortality before Age 60,Mortality between Age 15 and 50,Male Mortality between Age 15 and 50,Female Mortality between Age 15 and 50,Male Mortality between Age 15 and 60,Female Mortality between Age 15 and 60,Net Number of Migrants,Net Migration Rate
1,WORLD,NaN,World,1950,2499322.157,1245362.965,1253959.192,19.166,99.314,22.224,...,540.385,580.75,498.04,240.316,271.625,208.192,430.259,324.931,0,0


In [12]:
# Drop columns containing highlighted string for historical data
var = [('Male'), ('Female'), ('under')]
for i in var:
    df = df.drop(df.filter(regex=i).columns, axis=1)

In [13]:
# Drop columns containing highlighted string for estimated data
for i in var:
    es = es.drop(es.filter(regex=i).columns, axis=1)

In [14]:
# Subset datasets to Year to drop entries containing null values from datasets
df = df[df['Year'].notna()]
es = es[es['Year'].notna()]

In [15]:
# Manually rename columns in datasets
df = df.rename(columns={'Region, subregion, country or area *':'Region', 'Type':'Region Type', 
                        'Natural Change, Births minus Deaths':'Natural Change', 'ISO3 Alpha-code':'Country'})
es = es.rename(columns={'Region, subregion, country or area *':'Region', 'Type':'Region Type',
                        'Natural Change, Births minus Deaths':'Natural Change', 'ISO3 Alpha-code':'Country'})

In [16]:
# Drop rows with unspecified values connoted with ellipses
empty = df[df['Natural Change'] == '...'].index
df = df[~df['Natural Change'].index.isin(empty)]

empty = es[es['Natural Change'] == '...'].index
es = es[~es['Natural Change'].index.isin(empty)]

Since upper middle-income and lower middle-income countries are summed up as `middle-income countries`, we would drop the upper and lower middle entries from our datasets.

In [17]:
# Drop all columns of Lower-middle and Upper-middle income countries
df.drop(df.loc[df['Region'] == 'Lower-middle-income countries'].index, inplace=True)
df.drop(df.loc[df['Region'] == 'Upper-middle-income countries'].index, inplace=True)

Countries will be listed according to region type using their Alpha-3 codes.

In [18]:
# All countries in the world
World = ['ABW', 'AFG', 'AGO', 'AIA', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN',
          'BES', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLM', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA',
          'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COK', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CUW', 'CYM',
          'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESH', 'ESP', 'EST', 'ETH', 'FIN', 'FJI',
          'FLK', 'FRA', 'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GGY', 'GHA', 'GIB', 'GIN', 'GLP', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD',
          'GRL', 'GTM', 'GUF', 'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL',
          'ISR', 'ITA', 'JAM', 'JEY', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBR',
          'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAC', 'MAF', 'MAR', 'MCO', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL',
          'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MNP', 'MOZ', 'MRT', 'MSR', 'MTQ', 'MUS', 'MWI', 'MYS', 'MYT', 'NAM', 'NCL',
          'NER', 'NGA', 'NIC', 'NIU', 'NLD', 'NOR', 'NPL', 'NRU', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'POL',
          'PRI', 'PRK', 'PRT', 'PRY', 'PSE', 'PYF', 'QAT', 'REU', 'ROU', 'RUS', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SHN', 'SLB',
          'SLE', 'SLV', 'SMR', 'SOM', 'SPM', 'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 'SXM', 'SYC', 'SYR', 'TCA',
          'TCD', 'TGO', 'THA', 'TJK', 'TKL', 'TKM', 'TLS', 'TON', 'TTO', 'TUN', 'TUR', 'TUV', 'TWN', 'TZA', 'UGA', 'UKR', 'URY',
          'USA', 'UZB', 'VCT', 'VEN', 'VGB', 'VIR', 'VNM', 'VUT', 'WLF', 'WSM', 'XKX', 'YEM', 'ZAF', 'ZMB', 'ZWE']

In [19]:
# Countries in Africa
Western_Africa = ['BEN', 'BFA', 'CIV', 'CPV', 'GHA', 'GIN', 'GMB', 'GNB', 'LBR', 'MLI',
                  'MRT', 'NER', 'NGA', 'SEN', 'SHN', 'SLE', 'TGO']
Northern_Africa = ['DZA', 'EGY', 'ESH', 'LBY', 'MAR', 'SDN', 'TUN']
Eastern_Africa = ['BDI', 'COM', 'DJI', 'ERI', 'ETH', 'KEN', 'MDG', 'MOZ', 'MUS', 'MWI',
                  'MYT', 'REU', 'RWA', 'SOM', 'SSD', 'SYC', 'TZA', 'UGA', 'ZMB', 'ZWE']
Central_Africa = ['AGO', 'CAF', 'CMR', 'COD', 'COG', 'GAB', 'GNQ', 'STP', 'TCD']
Southern_Africa = ['BWA', 'LSO', 'NAM', 'SWZ', 'ZAF']

Africa = ['AGO', 'BDI', 'BEN', 'BFA', 'BWA', 'CAF', 'CIV', 'CMR', 'COD', 'COG', 'COM', 'CPV', 'DJI', 'DZA', 'EGY', 'ERI', 'ESH',
          'ETH', 'GAB', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'KEN', 'LBR', 'LBY', 'LSO', 'MAR', 'MDG', 'MLI', 'MOZ', 'MRT', 'MUS',
          'MWI', 'MYT', 'NAM', 'NER', 'NGA', 'REU', 'RWA', 'SDN', 'SEN', 'SHN', 'SLE', 'SOM', 'SSD', 'STP', 'SWZ', 'SYC', 'TCD',
          'TGO', 'TUN', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE']

In [20]:
# Countries in Asia
Central_Asia = ['KAZ', 'KGZ', 'TJK', 'TKM', 'UZB']
Eastern_Asia = ['CHN', 'HKG', 'JPN', 'KOR', 'MAC', 'MNG', 'PRK', 'TWN']
Southern_Asia = ['AFG', 'BGD', 'BTN', 'IND', 'IRN', 'LKA', 'MDV', 'NPL', 'PAK']
South_Eastern_Asia = ['BRN', 'IDN', 'KHM', 'LAO', 'MMR', 'MYS', 'PHL', 'SGP', 'THA', 'TLS', 'VNM']
Western_Asia = ['ARE', 'ARM', 'AZE', 'BHR', 'CYP', 'GEO', 'IRQ', 'ISR', 'JOR',
                'KWT', 'LBN', 'OMN', 'PSE', 'QAT', 'SAU', 'SYR', 'TUR', 'YEM']

Asia = ['AFG', 'ARE', 'ARM', 'AZE', 'BGD', 'BHR', 'BRN', 'BTN', 'CHN', 'CYP', 'GEO', 'HKG', 'IDN', 'IND', 'IRN', 'IRQ', 'ISR',
        'JOR', 'JPN', 'KAZ', 'KGZ', 'KHM', 'KOR', 'KWT', 'LAO', 'LBN', 'LKA', 'MAC', 'MDV', 'MMR', 'MNG', 'MYS', 'NPL', 'OMN',
        'PAK', 'PHL', 'PRK', 'PSE', 'QAT', 'SAU', 'SGP', 'SYR', 'THA', 'TJK', 'TKM', 'TLS', 'TUR', 'TWN', 'UZB', 'VNM', 'YEM']

In [21]:
# Countries in Europe
Eastern_Europe = ['BGR', 'BLR', 'CZE', 'HUN', 'MDA', 'POL', 'ROU', 'RUS', 'SVK', 'UKR']
Northern_Europe = ['DNK', 'EST', 'FIN', 'FRO', 'GBR', 'GGY', 'IMN', 'IRL', 'ISL', 'JEY', 'LTU', 'LVA', 'NOR', 'SWE']
Southern_Europe = ['ALB', 'AND', 'BIH', 'ESP', 'GIB', 'GRC', 'HRV', 'ITA', 'MKD', 'MLT', 'MNE', 'PRT', 'SMR', 'SRB', 'SVN', 'XKX']
Western_Europe = ['AUT', 'BEL', 'CHE', 'DEU', 'FRA', 'LIE', 'LUX', 'MCO', 'NLD']

Europe = ['ALB', 'AND', 'AUT', 'BEL', 'BGR', 'BIH', 'BLR', 'CHE', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'FRO', 'GBR',
          'GGY', 'GIB', 'GRC', 'HRV', 'HUN', 'IMN', 'IRL', 'ISL', 'ITA', 'JEY', 'LIE', 'LTU', 'LUX', 'LVA', 'MCO', 'MDA', 'MKD',
          'MLT', 'MNE', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SMR', 'SRB', 'SVK', 'SVN', 'SWE', 'UKR', 'XKX']

In [22]:
# Countries in Northern America
Northern_America = ['BMU', 'CAN', 'GRL', 'SPM', 'USA']

In [23]:
# Countries in Latin America & the Caribbean
Caribbean = ['ABW', 'AIA', 'ATG', 'BES', 'BHS', 'BLM', 'BRB', 'CUB', 'CUW', 'CYM', 'DMA', 'DOM', 'GLP', 'GRD', 'HTI',
             'JAM', 'KNA', 'LCA', 'MAF', 'MSR', 'MTQ', 'PRI', 'SXM', 'TCA', 'TTO', 'VCT', 'VGB', 'VIR']
Central_America = ['BLZ', 'CRI', 'GTM', 'HND', 'MEX', 'NIC', 'PAN', 'SLV']
South_America = ['ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'FLK', 'GUF', 'GUY', 'PER', 'PRY', 'SUR', 'URY', 'VEN']

Latin_America_and_the_Caribbean = ['ABW', 'AIA', 'ARG', 'ATG', 'BES', 'BHS', 'BLM', 'BLZ', 'BOL', 'BRA', 'BRB', 'CHL', 'COL',
                                   'CRI', 'CUB', 'CUW', 'CYM', 'DMA', 'DOM', 'ECU', 'FLK', 'GLP', 'GRD', 'GTM', 'GUF', 'GUY',
                                   'HND', 'HTI', 'JAM', 'KNA', 'LCA', 'MAF', 'MEX', 'MSR', 'MTQ', 'NIC', 'PAN', 'PER', 'PRI',
                                   'PRY', 'SLV', 'SUR', 'SXM', 'TCA', 'TTO', 'URY', 'VCT', 'VEN', 'VGB', 'VIR']

In [24]:
# Countries in Oceania
Australia_New_Zealand = ['AUS', 'NZL']
Melanesia = ['FJI', 'NCL', 'PNG', 'SLB', 'VUT']
Micronesia = ['FSM', 'GUM', 'KIR', 'MHL', 'MNP', 'NRU', 'PLW']
Polynesia = ['ASM', 'COK', 'NIU', 'PYF', 'TKL', 'TON', 'TUV', 'WLF', 'WSM']

Oceania = ['ASM', 'AUS', 'COK', 'FJI', 'FSM', 'GUM', 'KIR', 'MHL', 'MNP', 'NCL', 'NIU', 'NRU',
           'NZL', 'PLW', 'PNG', 'PYF', 'SLB', 'TKL', 'TON', 'TUV', 'VUT', 'WLF', 'WSM']

In [25]:
# Development groups
More_developed_countries = ['ALB', 'AND', 'AUS', 'AUT', 'BEL', 'BGR', 'BIH', 'BLR', 'BMU', 'CAN', 'CHE', 'CZE', 'DEU', 'DNK',
                            'ESP', 'EST', 'FIN', 'FRA', 'FRO', 'GBR', 'GGY', 'GIB', 'GRC', 'GRL', 'HRV', 'HUN', 'IMN', 'IRL',
                            'ISL', 'ITA', 'JEY', 'JPN', 'LIE', 'LTU', 'LUX', 'LVA', 'MCO', 'MDA', 'MKD', 'MLT', 'MNE', 'NLD',
                            'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SMR', 'SPM', 'SRB', 'SVK', 'SVN', 'SWE', 'UKR', 'USA',
                            'XKX']
Less_developed_countries = ['ABW', 'AFG', 'AGO', 'AIA', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AZE', 'BDI', 'BEN', 'BES', 'BFA',
                            'BGD', 'BHR', 'BHS', 'BLM', 'BLZ', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CHL', 'CHN',
                            'CIV', 'CMR', 'COD', 'COG', 'COK', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CUW', 'CYM', 'CYP', 'DJI',
                            'DMA', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESH', 'ETH', 'FJI', 'FLK', 'FSM', 'GAB', 'GEO', 'GHA',
                            'GIN', 'GLP', 'GMB', 'GNB', 'GNQ', 'GRD', 'GTM', 'GUF', 'GUM', 'GUY', 'HKG', 'HND', 'HTI', 'IDN', 
                            'IND', 'IRN', 'IRQ', 'ISR', 'JAM', 'JOR', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT',
                            'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO', 'MAC', 'MAF', 'MAR', 'MDG', 'MDV', 'MEX', 'MHL',
                            'MLI', 'MMR', 'MNG', 'MNP', 'MOZ', 'MRT', 'MSR', 'MTQ', 'MUS', 'MWI', 'MYS', 'MYT', 'NAM', 'NCL',
                            'NER', 'NGA', 'NIC', 'NIU', 'NPL', 'NRU', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'PRI',
                            'PRK', 'PRY', 'PSE', 'PYF', 'QAT', 'REU', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SHN', 'SLB', 'SLE',
                            'SLV', 'SOM', 'SSD', 'STP', 'SUR', 'SWZ', 'SXM', 'SYC', 'SYR', 'TCA', 'TCD', 'TGO', 'THA', 'TJK',
                            'TKL', 'TKM', 'TLS', 'TON', 'TTO', 'TUN', 'TUR', 'TUV', 'TWN', 'TZA', 'UGA', 'URY', 'UZB', 'VCT',
                            'VEN', 'VGB', 'VIR', 'VNM', 'VUT', 'WLF', 'WSM', 'YEM', 'ZAF', 'ZMB', 'ZWE']
Least_developed_countries = ['AFG', 'AGO', 'BDI', 'BEN', 'BFA', 'BGD', 'BTN', 'CAF', 'COD', 'COM', 'DJI', 'ERI', 'ETH', 'GIN',
                             'GMB', 'GNB', 'HTI', 'KHM', 'KIR', 'LAO', 'LBR', 'LSO', 'MDG', 'MLI', 'MMR', 'MOZ', 'MRT', 'MWI',
                             'NER', 'NPL', 'RWA', 'SDN', 'SEN', 'SLB', 'SLE', 'SOM', 'SSD', 'STP', 'TCD', 'TGO', 'TLS', 'TUV',
                             'TZA', 'UGA', 'YEM', 'ZMB']

In [26]:
# Less developed countries with exclusions
Less_developed_excluding_least_developed = ['ABW', 'AIA', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AZE', 'BES', 'BHR', 'BHS', 'BLM',
                                            'BLZ', 'BOL', 'BRA', 'BRB', 'BRN', 'BWA', 'CHL', 'CHN', 'CIV', 'CMR', 'COG', 'COK',
                                            'COL', 'CPV', 'CRI', 'CUB', 'CUW', 'CYM', 'CYP', 'DMA', 'DOM', 'DZA', 'ECU', 'EGY',
                                            'ESH', 'FJI', 'FLK', 'FSM', 'GAB', 'GEO', 'GHA', 'GLP', 'GNQ', 'GRD', 'GTM', 'GUF',
                                            'GUM', 'GUY', 'HKG', 'HND', 'IDN', 'IND', 'IRN', 'IRQ', 'ISR', 'JAM', 'JOR', 'KAZ',
                                            'KEN', 'KGZ', 'KNA', 'KOR', 'KWT', 'LBN', 'LBY', 'LCA', 'LKA', 'MAC', 'MAF', 'MAR',
                                            'MDV', 'MEX', 'MHL', 'MNG', 'MNP', 'MSR', 'MTQ', 'MUS', 'MYS', 'MYT', 'NAM', 'NCL',
                                            'NGA', 'NIC', 'NIU', 'NRU', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'PRI',
                                            'PRK', 'PRY', 'PSE', 'PYF', 'QAT', 'REU', 'SAU', 'SGP', 'SHN', 'SLV', 'SUR', 'SWZ',
                                            'SXM', 'SYC', 'SYR', 'TCA', 'THA', 'TJK', 'TKL', 'TKM', 'TON', 'TTO', 'TUN', 'TUR',
                                            'TWN', 'URY', 'UZB', 'VCT', 'VEN', 'VGB', 'VIR', 'VNM', 'VUT', 'WLF', 'WSM', 'ZAF',
                                            'ZWE']
# Excluding China
Less_developed_excluding_China = ['ABW', 'AFG', 'AGO', 'AIA', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AZE', 'BDI', 'BEN', 'BES',
                                  'BFA', 'BGD', 'BHR', 'BHS', 'BLM', 'BLZ', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF',
                                  'CHL', 'CIV', 'CMR', 'COD', 'COG', 'COK', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CUW', 'CYM',
                                  'CYP', 'DJI', 'DMA', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESH', 'ETH', 'FJI', 'FLK', 'FSM',
                                  'GAB', 'GEO', 'GHA', 'GIN', 'GLP', 'GMB', 'GNB', 'GNQ', 'GRD', 'GTM', 'GUF', 'GUM', 'GUY',
                                  'HKG', 'HND', 'HTI', 'IDN', 'IND', 'IRN', 'IRQ', 'ISR', 'JAM', 'JOR', 'KAZ', 'KEN', 'KGZ',
                                  'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO', 'MAC',
                                  'MAF', 'MAR', 'MDG', 'MDV', 'MEX', 'MHL', 'MLI', 'MMR', 'MNG', 'MNP', 'MOZ', 'MRT', 'MSR',
                                  'MTQ', 'MUS', 'MWI', 'MYS', 'MYT', 'NAM', 'NCL', 'NER', 'NGA', 'NIC', 'NIU', 'NPL', 'NRU',
                                  'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'PRI', 'PRK', 'PRY', 'PSE', 'PYF', 'QAT',
                                  'REU', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SHN', 'SLB', 'SLE', 'SLV', 'SOM', 'SSD', 'STP',
                                  'SUR', 'SWZ', 'SXM', 'SYC', 'SYR', 'TCA', 'TCD', 'TGO', 'THA', 'TJK', 'TKL', 'TKM', 'TLS',
                                  'TON', 'TTO', 'TUN', 'TUR', 'TUV', 'TWN', 'TZA', 'UGA', 'URY', 'UZB', 'VCT', 'VEN', 'VGB',
                                  'VIR', 'VNM', 'VUT', 'WLF', 'WSM', 'YEM', 'ZAF', 'ZMB', 'ZWE']

In [27]:
# SDG groups
Sub_Saharan_Africa = ['AGO', 'BDI', 'BEN', 'BFA', 'BWA', 'CAF', 'CIV', 'CMR', 'COD', 'COG', 'COM', 'CPV', 'DJI', 'ERI', 'ETH',
                      'GAB', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'KEN', 'LBR', 'LSO', 'MDG', 'MLI', 'MOZ', 'MRT', 'MUS', 'MWI',
                      'MYT', 'NAM', 'NER', 'NGA', 'REU', 'RWA', 'SEN', 'SHN', 'SLE', 'SOM', 'SSD', 'STP', 'SWZ', 'SYC', 'TCD',
                      'TGO', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE']
SDG_Northern_Africa_and_Western_Asia = ['ARE', 'ARM', 'AZE', 'BHR', 'CYP', 'DZA', 'EGY', 'ESH', 'GEO', 'IRQ', 'ISR', 'JOR',
                                        'KWT', 'LBN', 'LBY', 'MAR', 'OMN', 'PSE', 'QAT', 'SAU', 'SDN', 'SYR', 'TUN', 'TUR',
                                        'YEM']
SDG_Central_and_Southern_Asia = ['AFG', 'BGD', 'BTN', 'IND', 'IRN', 'KAZ', 'KGZ', 'LKA', 'MDV', 'NPL', 'PAK', 'TJK', 'TKM', 'UZB']
SDG_Eastern_and_South_Eastern_Asia = ['BRN', 'CHN', 'HKG', 'IDN', 'JPN', 'KHM', 'KOR', 'LAO', 'MAC', 'MMR', 'MNG', 'MYS', 'PHL',
                                      'PRK', 'SGP', 'THA', 'TLS', 'TWN', 'VNM']
SDG_Oceania = ['ASM', 'COK', 'FJI', 'FSM', 'GUM', 'KIR', 'MHL', 'MNP', 'NCL', 'NIU', 'NRU', 'PLW', 'PNG', 'PYF', 'SLB', 'TKL',
               'TON', 'TUV', 'VUT', 'WLF', 'WSM']
SDG_Latin_America_and_the_Caribbean = ['ABW', 'AIA', 'ARG', 'ATG', 'BES', 'BHS', 'BLM', 'BLZ', 'BOL', 'BRA', 'BRB', 'CHL', 'COL',
                                       'CRI', 'CUB', 'CUW', 'CYM', 'DMA', 'DOM', 'ECU', 'FLK', 'GLP', 'GRD', 'GTM', 'GUF', 'GUY',
                                       'HND', 'HTI', 'JAM', 'KNA', 'LCA', 'MAF', 'MEX', 'MSR', 'MTQ', 'NIC', 'PAN', 'PER', 'PRI',
                                       'PRY', 'SLV', 'SUR', 'SXM', 'TCA', 'TTO', 'URY', 'VCT', 'VEN', 'VGB', 'VIR']
SDG_Australia_New_Zealand = ['AUS', 'NZL']
SDG_Europe_and_Northern_America = ['ALB', 'AND', 'AUT', 'BEL', 'BGR', 'BIH', 'BLR', 'BMU', 'CAN', 'CHE', 'CZE', 'DEU', 'DNK',
                                   'ESP', 'EST', 'FIN', 'FRA', 'FRO', 'GBR', 'GGY', 'GIB', 'GRC', 'GRL', 'HRV', 'HUN', 'IMN',
                                   'IRL', 'ISL', 'ITA', 'JEY', 'LIE', 'LTU', 'LUX', 'LVA', 'MCO', 'MDA', 'MKD', 'MLT', 'MNE',
                                   'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SMR', 'SPM', 'SRB', 'SVK', 'SVN', 'SWE', 'UKR',
                                   'USA', 'XKX']

In [28]:
# Income groups
Low_income_countries = ['AFG', 'BDI', 'BFA', 'CAF', 'COD', 'ERI', 'ETH', 'GIN', 'GMB', 'GNB', 'LBR', 'MDG', 'MLI', 'MOZ',
                        'MWI', 'NER', 'PsRK', 'RWA', 'SDN', 'SLE', 'SOM', 'SSD', 'SYR', 'TCD', 'TGO', 'UGA', 'YEM', 'ZMB']
Low_middle_income_countries = ['AGO', 'BEN', 'BGD', 'BOL', 'BTN', 'CIV', 'CMR', 'COG', 'COM', 'CPV', 'DJI', 'DZA', 'EGY', 'FSM',
                               'GHA', 'HND', 'HTI', 'IDN', 'IND', 'IRN', 'KEN', 'KGZ', 'KHM', 'KIR', 'LAO', 'LBN', 'LKA', 'LSO',
                               'MAR', 'MMR', 'MNG', 'MRT', 'NGA', 'NIC', 'NPL', 'PAK', 'PHL', 'PNG', 'PRK', 'PSE', 'SEN', 'SLB',
                               'SLV', 'STP', 'SWZ', 'TJK', 'TLS', 'TUN', 'TZA', 'UKR', 'UZB', 'VNM', 'VUT', 'WSM', 'ZWE']
Upper_middle_income_countries = ['ALB', 'ARG', 'ARM', 'ASM', 'AZE', 'BGR', 'BIH', 'BLR', 'BLZ', 'BRA', 'BWA', 'CHN', 'COL',
                                 'CRI', 'CUB', 'DMA', 'DOM', 'ECU', 'FJI', 'GAB', 'GEO', 'GNQ', 'GRD', 'GTM', 'GUY', 'IRQ',
                                 'JAM', 'JOR', 'KAZ', 'LBY', 'LCA', 'MDA', 'MDV', 'MEX', 'MHL', 'MKD', 'MNE', 'MUS', 'MYS',
                                 'NAM', 'PER', 'PLW', 'PRY', 'RUS', 'SRB', 'SUR', 'THA', 'TKM', 'TON', 'TUR', 'TUV', 'VCT',
                                 'XKX', 'ZAF']
Middle_income_countries = ['AGO', 'ALB', 'ARG', 'ARM', 'ASM', 'AZE', 'BEN', 'BGD', 'BGR', 'BIH', 'BLR', 'BLZ', 'BOL', 'BRA',
                           'BTN', 'BWA', 'CHN', 'CIV', 'CMR', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'DJI', 'DMA', 'DOM',
                           'DZA', 'ECU', 'EGY', 'FJI', 'FSM', 'GAB', 'GEO', 'GHA', 'GNQ', 'GRD', 'GTM', 'GUY', 'HND', 'HTI',
                           'IDN', 'IND', 'IRN', 'IRQ', 'JAM', 'JOR', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'LAO', 'LBN', 'LBY',
                           'LCA', 'LKA', 'LSO', 'MAR', 'MDA', 'MDV', 'MEX', 'MHL', 'MKD', 'MMR', 'MNE', 'MNG', 'MRT', 'MUS',
                           'MYS', 'NAM', 'NGA', 'NIC', 'NPL', 'PAK', 'PER', 'PHL', 'PLW', 'PNG', 'PRK', 'PRY', 'PSE', 'RUS',
                           'SEN', 'SLB', 'SLV', 'SRB', 'STP', 'SUR', 'SWZ', 'THA', 'TJK', 'TKM', 'TLS', 'TON', 'TUN', 'TUR',
                           'TUV', 'TZA', 'UKR', 'UZB', 'VCT', 'VNM', 'VUT', 'WSM', 'XKX', 'ZAF', 'ZWE']
High_income_countries = ['ABW', 'AND', 'ARE', 'ATG', 'AUS', 'AUT', 'BEL', 'BHR', 'BHS', 'BMU', 'BRB', 'BRN', 'CAN', 'CHE', 'CHL',
                         'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'FRO', 'GBR', 'GGY', 'GIB', 'GRC',
                         'GRL', 'GUM', 'HKG', 'HRV', 'HUN', 'IMN', 'IRL', 'ISL', 'ISR', 'ITA', 'JEY', 'JPN', 'KNA', 'KOR', 'KWT',
                         'LIE', 'LTU', 'LUX', 'LVA', 'MAC', 'MAF', 'MCO', 'MLT', 'MNP', 'NCL', 'NLD', 'NOR', 'NRU', 'NZL', 'OMN',
                         'PAN', 'POL', 'PRI', 'PRT', 'PYF', 'QAT', 'ROU', 'SAU', 'SGP', 'SMR', 'SVK', 'SVN', 'SWE', 'SXM', 'SYC',
                         'TCA', 'TTO', 'TWN', 'URY', 'USA', 'VGB', 'VIR']
No_income_group = ['AIA', 'BES', 'BLM', 'COK', 'ESH', 'FLK', 'GLP', 'GUF', 'MSR', 'MTQ', 'MYT', 'NIU', 'REU',
                   'SHN', 'SPM', 'TKL', 'VEN', 'WLF']

In [29]:
# Other special groups
Landlocked_developing_countries = ['AFG', 'ARM', 'AZE', 'BDI', 'BFA', 'BOL', 'BTN', 'BWA', 'CAF', 'ETH', 'KAZ', 'KGZ', 'LAO',
                                   'LSO', 'MDA', 'MKD', 'MLI', 'MNG', 'MWI', 'NER', 'NPL', 'PRY', 'RWA', 'SSD', 'SWZ', 'TCD',
                                   'TJK', 'TKM', 'UGA', 'UZB', 'ZMB', 'ZWE']
Small_island_developing_countries = ['ABW', 'AIA', 'ASM', 'ATG', 'BHR', 'BHS', 'BLZ', 'BMU', 'BRB', 'COK', 'COM', 'CPV', 'CUB',
                                     'CUW', 'CYM', 'DMA', 'DOM', 'FJI', 'FSM', 'GLP', 'GNB', 'GRD', 'GUM', 'GUY', 'HTI', 'JAM',
                                     'KIR', 'KNA', 'LCA', 'MDV', 'MHL', 'MNP', 'MSR', 'MTQ', 'MUS', 'NCL', 'NIU', 'NRU', 'PLW',
                                     'PNG', 'PRI', 'PYF', 'SGP', 'SLB', 'STP', 'SUR', 'SXM', 'SYC', 'TCA', 'TLS', 'TON', 'TTO',
                                     'TUV', 'VCT', 'VGB', 'VIR', 'VUT', 'WSM']

In [30]:
# Convert datatypes of 'Region Type' and 'Year' to category and period respectively for historical data
df['Region Type'] = df['Region Type'].astype('category')
df['Year'] = pd.to_datetime(df['Year'], format='%Y').dt.to_period('Y')

# Convert datatypes of 'Region Type' and 'Year' to category and period  for estimated data
es['Region Type'] = es['Region Type'].astype('category')
es['Year'] = pd.to_datetime(es['Year'], format='%Y').dt.to_period('Y')

In [31]:
# Convert datatypes of unspecified columns to floats for historical data
others = ['Region', 'Region Type', 'Year', 'Country']
cols = [i for i in df.columns if i not in others]
for col in cols:
    df[col] = df[col].apply(pd.to_numeric, errors='coerce')

In [32]:
# Convert datatypes of unspecified columns to floats for estimated data
for col in cols:
    es[col] = es[col].apply(pd.to_numeric, errors='coerce')

In [33]:
# Multiply numeric values of selected columns by 1000
num_vars = ['Total Population', 'Natural Change', 'Population Change', 'Births', 'Births by women aged 15 to 19',
            'Total Deaths', 'Live Births Surviving to Age 1', 'Net Number of Migrants']
df[num_vars] *= 1000
es[num_vars] *= 1000

In [34]:
# Reset index
df = df.reset_index()
es = es.reset_index()

In [35]:
# Subset dataframes by country
df_2 = df[df['Region Type'] == 'Country/Area']
es_2 = es[es['Region Type'] == 'Country/Area']

In [36]:
# Create new copy of subsetted data
df_ = df_2.copy()
es_ = es_2.copy()

In [37]:
# Drop Region and Region type columns
df_.drop(['Region', 'Region Type'], axis=1, inplace=True)
es_.drop(['Region', 'Region Type'], axis=1, inplace=True)

In [38]:
# Rename country column
df_ = df_.rename(columns={'Country':'Country/Area'})
es_ = es_.rename(columns={'Country':'Country/Area'})

In [39]:
# Create empty columns of region types in both datasets to be filled with list of string values
df_['Region'] = ''
df_['Subregion'] = ''
df_['Development Group'] = ''
df_['Income Group'] = ''
df_['SDG region'] = ''
df_['Special other'] = ''

es_['Region'] = ''
es_['Subregion'] = ''
es_['Development Group'] = ''
es_['Income Group'] = ''
es_['SDG region'] = ''
es_['Special other'] = ''

In [40]:
# Assign Regions (historical)
df_.loc[df_['Country/Area'].isin(Africa),'Region'] = 'AFRICA'
df_.loc[df_['Country/Area'].isin(Asia),'Region'] = 'ASIA'
df_.loc[df_['Country/Area'].isin(Europe),'Region'] = 'EUROPE'
df_.loc[df_['Country/Area'].isin(Latin_America_and_the_Caribbean),'Region'] = 'LATIN AMERICA AND THE CARIBBEAN'
df_.loc[df_['Country/Area'].isin(Northern_America),'Region'] = 'NORTHERN AMERICA'
df_.loc[df_['Country/Area'].isin(Oceania),'Region'] = 'OCEANIA'

In [41]:
# Assign Regions (estimated)
es_.loc[es_['Country/Area'].isin(Africa),'Region'] = 'AFRICA'
es_.loc[es_['Country/Area'].isin(Asia),'Region'] = 'ASIA'
es_.loc[es_['Country/Area'].isin(Europe),'Region'] = 'EUROPE'
es_.loc[es_['Country/Area'].isin(Latin_America_and_the_Caribbean),'Region'] = 'LATIN AMERICA AND THE CARIBBEAN'
es_.loc[es_['Country/Area'].isin(Northern_America),'Region'] = 'NORTHERN AMERICA'
es_.loc[es_['Country/Area'].isin(Oceania),'Region'] = 'OCEANIA'

In [42]:
# Assign Subregions (historical)
df_.loc[df_['Country/Area'].isin(Eastern_Africa),'Subregion'] = 'Eastern Africa'
df_.loc[df_['Country/Area'].isin(Central_Africa),'Subregion'] = 'Middle Africa'
df_.loc[df_['Country/Area'].isin(Northern_Africa),'Subregion'] = 'Northern Africa'
df_.loc[df_['Country/Area'].isin(Southern_Africa),'Subregion'] = 'Southern Africa'
df_.loc[df_['Country/Area'].isin(Western_Africa),'Subregion'] = 'Western Africa'
df_.loc[df_['Country/Area'].isin(Central_Asia),'Subregion'] = 'Central Asia'
df_.loc[df_['Country/Area'].isin(Eastern_Asia),'Subregion'] = 'Eastern Asia'
df_.loc[df_['Country/Area'].isin(Southern_Asia),'Subregion'] = 'Southern Asia'
df_.loc[df_['Country/Area'].isin(South_Eastern_Asia),'Subregion'] = 'South-Eastern Asia'
df_.loc[df_['Country/Area'].isin(Western_Asia),'Subregion'] = 'Western Asia'
df_.loc[df_['Country/Area'].isin(Eastern_Europe),'Subregion'] = 'Eastern Europe'
df_.loc[df_['Country/Area'].isin(Northern_Europe),'Subregion'] = 'Northern Europe'
df_.loc[df_['Country/Area'].isin(Southern_Europe),'Subregion'] = 'Southern Europe'
df_.loc[df_['Country/Area'].isin(Western_Europe),'Subregion'] = 'Western Europe'
df_.loc[df_['Country/Area'].isin(Northern_America),'Subregion'] = 'Northern America'
df_.loc[df_['Country/Area'].isin(Caribbean),'Subregion'] = 'Caribbean'
df_.loc[df_['Country/Area'].isin(Central_America),'Subregion'] = 'Central America'
df_.loc[df_['Country/Area'].isin(South_America),'Subregion'] = 'South America'
df_.loc[df_['Country/Area'].isin(Australia_New_Zealand),'Subregion'] = 'Australia/New Zealand'
df_.loc[df_['Country/Area'].isin(Melanesia),'Subregion'] = 'Melanesia'
df_.loc[df_['Country/Area'].isin(Micronesia),'Subregion'] = 'Micronesia'
df_.loc[df_['Country/Area'].isin(Polynesia),'Subregion'] = 'Polynesia'

In [43]:
# Assign Subregions (estimated)
es_.loc[es_['Country/Area'].isin(Eastern_Africa),'Subregion'] = 'Eastern Africa'
es_.loc[es_['Country/Area'].isin(Central_Africa),'Subregion'] = 'Middle Africa'
es_.loc[es_['Country/Area'].isin(Northern_Africa),'Subregion'] = 'Northern Africa'
es_.loc[es_['Country/Area'].isin(Southern_Africa),'Subregion'] = 'Southern Africa'
es_.loc[es_['Country/Area'].isin(Western_Africa),'Subregion'] = 'Western Africa'
es_.loc[es_['Country/Area'].isin(Central_Asia),'Subregion'] = 'Central Asia'
es_.loc[es_['Country/Area'].isin(Eastern_Asia),'Subregion'] = 'Eastern Asia'
es_.loc[es_['Country/Area'].isin(Southern_Asia),'Subregion'] = 'Southern Asia'
es_.loc[es_['Country/Area'].isin(South_Eastern_Asia),'Subregion'] = 'South-Eastern Asia'
es_.loc[es_['Country/Area'].isin(Western_Asia),'Subregion'] = 'Western Asia'
es_.loc[es_['Country/Area'].isin(Eastern_Europe),'Subregion'] = 'Eastern Europe'
es_.loc[es_['Country/Area'].isin(Northern_Europe),'Subregion'] = 'Northern Europe'
es_.loc[es_['Country/Area'].isin(Southern_Europe),'Subregion'] = 'Southern Europe'
es_.loc[es_['Country/Area'].isin(Western_Europe),'Subregion'] = 'Western Europe'
es_.loc[es_['Country/Area'].isin(Northern_America),'Subregion'] = 'Northern America'
es_.loc[es_['Country/Area'].isin(Caribbean),'Subregion'] = 'Caribbean'
es_.loc[es_['Country/Area'].isin(Central_America),'Subregion'] = 'Central America'
es_.loc[es_['Country/Area'].isin(South_America),'Subregion'] = 'South America'
es_.loc[es_['Country/Area'].isin(Australia_New_Zealand),'Subregion'] = 'Australia/New Zealand'
es_.loc[es_['Country/Area'].isin(Melanesia),'Subregion'] = 'Melanesia'
es_.loc[es_['Country/Area'].isin(Micronesia),'Subregion'] = 'Micronesia'
es_.loc[es_['Country/Area'].isin(Polynesia),'Subregion'] = 'Polynesia'

In [44]:
# Assign Development groups (historical)
df_.loc[df_['Country/Area'].isin(More_developed_countries),'Development Group'] = 'More developed regions'
df_.loc[df_['Country/Area'].isin(Less_developed_countries),'Development Group'] = 'Less developed regions'

In [45]:
# Assign Development groups (estimated)
es_.loc[es_['Country/Area'].isin(More_developed_countries),'Development Group'] = 'More developed regions'
es_.loc[es_['Country/Area'].isin(Less_developed_countries),'Development Group'] = 'Less developed regions'

In [46]:
# Assign Income groups (historical)
df_.loc[df_['Country/Area'].isin(High_income_countries),'Income Group'] = 'High-income countries'
df_.loc[df_['Country/Area'].isin(Low_income_countries),'Income Group'] = 'Low-income countries'
df_.loc[df_['Country/Area'].isin(Middle_income_countries),'Income Group'] = 'Middle-income countries'
df_.loc[df_['Country/Area'].isin(No_income_group),'Income Group'] = 'No income group available'

In [47]:
# Assign Income groups (estimated)
es_.loc[es_['Country/Area'].isin(High_income_countries),'Income Group'] = 'High-income countries'
es_.loc[es_['Country/Area'].isin(Low_income_countries),'Income Group'] = 'Low-income countries'
es_.loc[es_['Country/Area'].isin(Middle_income_countries),'Income Group'] = 'Middle-income countries'
es_.loc[es_['Country/Area'].isin(No_income_group),'Income Group'] = 'No income group available'

In [48]:
# Assign SDG regions (historical)
df_.loc[df_['Country/Area'].isin(Sub_Saharan_Africa),'SDG region'] = 'Sub-Saharan Africa'
df_.loc[df_['Country/Area'].isin(SDG_Northern_Africa_and_Western_Asia),'SDG region'] = 'Northern Africa and Western Asia'
df_.loc[df_['Country/Area'].isin(SDG_Central_and_Southern_Asia),'SDG region'] = 'Central and Southern Asia'
df_.loc[df_['Country/Area'].isin(SDG_Eastern_and_South_Eastern_Asia),'SDG region'] = 'Eastern and South-Eastern Asia'
df_.loc[df_['Country/Area'].isin(SDG_Latin_America_and_the_Caribbean),'SDG region'] = 'Latin America and the Caribbean'
df_.loc[df_['Country/Area'].isin(SDG_Oceania),'SDG region'] = 'Oceania (excluding Australia and New Zealand)'
df_.loc[df_['Country/Area'].isin(SDG_Australia_New_Zealand),'SDG region'] = 'Australia/New Zealand'
df_.loc[df_['Country/Area'].isin(SDG_Europe_and_Northern_America),'SDG region'] = 'Europe and Northern America'

In [49]:
# Assign SDG regions (estimated)
es_.loc[es_['Country/Area'].isin(Sub_Saharan_Africa),'SDG region'] = 'Sub-Saharan Africa'
es_.loc[es_['Country/Area'].isin(SDG_Northern_Africa_and_Western_Asia),'SDG region'] = 'Northern Africa and Western Asia'
es_.loc[es_['Country/Area'].isin(SDG_Central_and_Southern_Asia),'SDG region'] = 'Central and Southern Asia'
es_.loc[es_['Country/Area'].isin(SDG_Eastern_and_South_Eastern_Asia),'SDG region'] = 'Eastern and South-Eastern Asia'
es_.loc[es_['Country/Area'].isin(SDG_Latin_America_and_the_Caribbean),'SDG region'] = 'Latin America and the Caribbean'
es_.loc[es_['Country/Area'].isin(SDG_Oceania),'SDG region'] = 'Oceania (excluding Australia and New Zealand)'
es_.loc[es_['Country/Area'].isin(SDG_Australia_New_Zealand),'SDG region'] = 'Australia/New Zealand'
es_.loc[es_['Country/Area'].isin(SDG_Europe_and_Northern_America),'SDG region'] = 'Europe and Northern America'

In [50]:
# Assign other special groups (historical)
df_.loc[df_['Country/Area'].isin(Landlocked_developing_countries),'Special other'] = 'Land-locked Developing Countries (LLDC)'
df_.loc[df_['Country/Area'].isin(Small_island_developing_countries),'Special other'] = 'Small Island Developing States (SIDS)'
Special = Landlocked_developing_countries + Small_island_developing_countries
df_.loc[~df_['Country/Area'].isin(Special),'Special other'] = 'Non-designated'

In [51]:
# Assign other special groups
es_.loc[es_['Country/Area'].isin(Landlocked_developing_countries),'Special other'] = 'Land-locked Developing Countries (LLDC)'
es_.loc[es_['Country/Area'].isin(Small_island_developing_countries),'Special other'] = 'Small Island Developing States (SIDS)'
es_.loc[~es_['Country/Area'].isin(Special),'Special other'] = 'Non-designated'

In [52]:
# Check for null values in datasets
print(df_.isnull().sum().any())
print(es_.isnull().sum().any())

False
False


In [53]:
# Create dictionary containing alpha 3 country codes as keys and country names as values
keys = df[df['Region Type'] == 'Country/Area'].Country.unique()
values = df[df['Region Type'] == 'Country/Area'].Region.unique()
dictionary = dict(zip(keys, values))

In [54]:
# Replace alpha 3 codes with country names
df_['Country/Area'] = df_['Country/Area'].replace(dictionary, regex=True)
es_['Country/Area'] = es_['Country/Area'].replace(dictionary, regex=True)

In [55]:
# Check dimensions
es_.shape, df_.shape

((18644, 38), (16992, 38))

In [ ]:
# Save tidy datasets
df_.to_csv('WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_REV1_past_comb.csv', index=False, encoding='utf-8')
es_.to_csv('WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_REV1_estimated_comb.csv', index=False, encoding='utf-8')